In [ ]:
from embedding_search.academic_analytics import get_units
from api.core import get_author_by_id, Engine
from embedding_search.vector_store import connect_milvus
from pymilvus import Collection
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
def get_default_engine():
    """Get a default engine with default collections."""

    connect_milvus()

    author_collection = Collection("authors")
    article_collection = Collection("articles")
    embeddings = OpenAIEmbeddings()

    return Engine(author_collection, article_collection, embeddings)

In [ ]:
engine = get_default_engine()

In [ ]:
y1 = engine.search_authors("mushroom and farming", top_k=3)
y1

In [ ]:
y2 = engine.search_authors("mushroom and farming", top_k=3, filter_unit="28626")
y2